<a href="https://colab.research.google.com/github/SohamSarkar06/stock-prediction/blob/main/stockshare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flask flask_cors yfinance scikit-learn pyngrok --quiet

In [3]:
!ngrok config add-authtoken 30MEuniNd0CwQRwyywZW0mZZ6Io_6GMJxLWU8wBb8tJc6EjHC



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
!pip install flask flask-cors yfinance ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.5 MB/s eta 0:00:00


In [17]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression

app = Flask(__name__)
CORS(app)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        symbol = data.get('symbol', '').upper()

        if not symbol:
            return jsonify({'error': 'No symbol provided'}), 400

        # Append .NS for Indian stocks
        yf_symbol = symbol + ".NS"
        end = datetime.now()
        start = end - timedelta(days=365)

        df = yf.download(yf_symbol, start=start.strftime("%Y-%m-%d"), end=end.strftime("%Y-%m-%d"))

        if df.empty or 'Close' not in df.columns:
            return jsonify({'error': 'Stock data not available.'})

        df = df[['Close']].dropna()
        df['Days'] = (df.index - df.index[0]).days
        X = np.array(df['Days']).reshape(-1, 1)
        y = np.array(df['Close'])

        model = LinearRegression()
        model.fit(X, y)

        future_days = np.array(range(X[-1][0] + 1, X[-1][0] + 31)).reshape(-1, 1)
        future_preds = model.predict(future_days)

        future_dates = [(df.index[0] + timedelta(days=int(day[0]))).strftime('%Y-%m-%d') for day in future_days]
        last_price = float(df['Close'].iloc[-1])

        return jsonify({
            'symbol': symbol,
            'last_price': last_price,
            'predicted_prices': list(map(float, future_preds)),
            'dates': future_dates
        })

    except Exception as e:
        return jsonify({'error': str(e)})

# ngrok integration to expose Colab app
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("Backend running at:", public_url)

app.run(port=5000)

Backend running at: NgrokTunnel: "https://1e41649be294.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Jul/2025 10:07:17] "OPTIONS /predict HTTP/1.1" 200 -
/tmp/ipython-input-17-1136737756.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(yf_symbol, start=start.strftime("%Y-%m-%d"), end=end.strftime("%Y-%m-%d"))
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GLOBAL.NS.NS']: YFTzMissingError('possibly delisted; no timezone found')
INFO:werkzeug:127.0.0.1 - - [25/Jul/2025 10:07:18] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jul/2025 10:08:09] "OPTIONS /predict HTTP/1.1" 200 -
/tmp/ipython-input-17-1136737756.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(y

In [18]:
!pip install prophet


In [ ]:

from flask import Flask, request, jsonify
import yfinance as yf
import pandas as pd
from prophet import Prophet
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.get_json()
        symbol = data.get("symbol", "").upper()
        if not symbol:
            return jsonify({"error": "No symbol provided."})

        # Add .NS suffix if not present
        if not symbol.endswith(".NS"):
            symbol += ".NS"

        df = yf.download(symbol, period="1y")
        if df.empty:
            return jsonify({"error": "No stock data found."})

        df = df.reset_index()
        df = df[["Date", "Close"]]
        df.columns = ["ds", "y"]

        model = Prophet()
        model.fit(df)

        future = model.make_future_dataframe(periods=30)
        forecast = model.predict(future)

        predicted = forecast[["ds", "yhat"]].tail(30)
        dates = predicted["ds"].dt.strftime("%Y-%m-%d").tolist()
        prices = predicted["yhat"].fillna(method='ffill').fillna(method='bfill').round(2).tolist()

        # Ensure the list is clean
        if not all(isinstance(p, (int, float)) for p in prices):
            return jsonify({"error": "Invalid predictions format."})

        last_price = df["y"].iloc[-1]

        return jsonify({
            "symbol": symbol,
            "last_price": round(float(last_price), 2),
            "dates": dates,
            "predicted_prices": prices
        })

    except Exception as e:
        return jsonify({"error": str(e)})


# ✅ ngrok block must be OUTSIDE the function
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("Backend running at:", public_url)

app.run(port=5000)


Backend running at: NgrokTunnel: "https://a44f3ecbd777.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Jul/2025 10:51:09] "OPTIONS /predict HTTP/1.1" 200 -
/tmp/ipython-input-28-3371363289.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="1y")
[*********************100%***********************]  1 of 1 completed
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkszh9oid/w0z4it30.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpkszh9oid/1b8ed9qy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/sta